# Socket

In [ ]:
import select
import socket
import os
import sys
import struct
import time

## IO多路复用

In [ ]:
# client 客户端
obj = socket.socket()

obj.connect(('127.0.0.1', 8801))

while True:
    inp = input("Please(q\退出):\n>>>")
    obj.sendall(bytes(inp, encoding="utf-8"))
    if inp == "q":
        break
    ret = str(obj.recv(1024), encoding="utf-8")
    print(ret)

In [ ]:
# 服务端 service
sk1 = socket.socket()
sk1.bind(("127.0.0.1", 8001))
sk1.listen()
inputs = [sk1]

message_dic = {}
outputs = []
while True:
    r_list, w_list, e_list = select.select(inputs, [], inputs, 1)
    if r_list:
        print("正在监听的socket对象%d" % len(inputs))
        print(r_list)

    for sk1_or_conn in r_list:
        if sk1_or_conn == sk1:
            conn, address = sk1_or_conn.accept()
            inputs.append(conn)
            message_dic[conn] = []
        else:
            try:
                data_bytes = sk1_or_conn.recv(1024)
                data_str = str(data_bytes, encoding="utf-8")
                sk1_or_conn.sendall(bytes(data_str + "好", encoding="utf-8"))
            except Exception as ex:
                inputs.remove(sk1_or_conn)
            else:
                data_str = str(data_bytes, encoding="utf-8")
                message_dic[sk1_or_conn].append(data_str)
                outputs.append(sk1_or_conn)
        for conn in w_list:
            recv_str = message_dic[conn][0]
            del message_dic[conn][0]
            conn.sendall(bytes(recv_str + "好", encoding="utf-8"))
        for sk in e_list:
            inputs.remove(sk)

## 文本传输

In [ ]:
# 客户端client
def sock_client_image():
    while True:
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            # 服务器和客户端在不同的系统或不同的主机下时使用的ip和端口，首先要查看服务器所在的系统网卡的ip
            # 服务器和客户端都在一个系统下时使用的ip和端口
            s.connect(('127.0.0.1', 8803))
        except socket.error as msg:
            print(msg)
            print(sys.exit(1))
        filepath = input('input the file: ')  # 输入当前目录下的图片名 xxx.jpg
        # 将xxx.jpg以128sq的格式打包
        fhead = struct.pack(b'128sq', bytes(os.path.basename(filepath), encoding='utf-8'), os.stat(filepath).st_size)
        s.send(fhead)

        fp = open(filepath, 'rb')  # 打开要传输的图片
        while True:
            data = fp.read(1024)  # 读入图片数据
            if not data:
                print('{0} send over...'.format(filepath))
                break
            s.send(data)  # 以二进制格式发送图片数据
            s.close()
        s.close()
        break    # 循环发送

In [ ]:
# 服务端service
def socket_service_image():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.bind(('127.0.0.1', 8803))
        s.listen(10)
    except socket.error as msg:
        print(msg)
        sys.exit(1)

    print("Wait for Connection.....................")

    while True:
        print(s.accept())
        sock, addr = s.accept()  # addr是一个元组(ip,port)
        deal_image(sock, addr)


def deal_image(sock, addr):
    print("Accept connection from {0}".format(addr))  # 查看发送端的ip和端口
    while True:
        fileinfosize = struct.calcsize('128sq')
        print('fileinfosize is', fileinfosize)
        buf = sock.recv(fileinfosize)  # 接收图片名
        print('buf is ', buf)
        if buf:
            filename, filesize = struct.unpack('128sq', buf)
            print('filename is {} ,filesize is {}'.format(filename.decode(), filesize))
            fn = filename.decode().strip('\x00')
            print('fn is ', fn)
            # 在服务器端新建图片名（可以不用新建的，直接用原来的也行，只要客户端和服务器不是同一个系统或接收到的图片和原图片不在一个文件夹下）
            new_filename = os.path.join('./', 'new_' + fn)

            recvd_size = 0
            fp = open(new_filename, 'wb')

            while not recvd_size == filesize:
                if filesize - recvd_size > 1024:
                    data = sock.recv(1024)
                    recvd_size += len(data)
                else:
                    data = sock.recv(1024)
                    recvd_size = filesize
                print('data is', data)
                fp.write(data)  # 写入图片数据
            fp.close()
        sock.close()
        break


if __name__ == '__main__':
    socket_service_image()

## 非阻塞

In [ ]:
# 客户端client
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# client.connect((socket.gethostbyname(socket.gethostname()), 8007))
client.connect(('127.0.0.1', 8807))

while True:
    cRequest = input("send: ")
    client.send(cRequest.encode())
    if cRequest == "quit":
        print("[+] Down line......")
        time.sleep(2)
        client.close()
        break

In [ ]:
# 服务端service
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.setblocking(False)  # 将socket设置为非阻塞. 在创建socket对象后就进行该操作.

# server.bind((socket.gethostbyname(socket.gethostname()), 8007))
server.bind(('127.0.0.1', 8807))
server.listen(5)

client_list = []

while True:
    try:
        connection, addr = server.accept()
        client_list.append((connection, addr))
        print("connected:{}".format(addr))

    # accept原本是阻塞的, 等待connect, 设置setblocking(False)后, accept不再阻塞,
    # 它会(不断的轮询)要求必须有connect来连接, 不然就引发BlockingIOError, 所以为了在没有connect时,
    # 我们捕捉这个异常并pass掉.
    except BlockingIOError:
        pass
    for client_socket, client_addr in client_list:
        try:
            client_recv = client_socket.recv(1024)
            if client_recv:
                print("receive:{}>>>\n{}".format(client_addr, client_recv.decode('utf-8')))
                client_socket.send(client_recv)

            else:
                client_socket.close()
                print("downline:{}".format(client_addr))
                client_list.remove((client_socket, client_addr))

        except (BlockingIOError, ConnectionResetError):
            pass